In [ ]:
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px  # for data visualization
import plotly.graph_objects as go  # for data visualization
from prophet import Prophet  # for time series forecasting

In [ ]:
data = pd.read_csv("data/data.csv")  # Read the original data from the csv file
data["Months"] = pd.to_datetime(
    data["Months"]
)  # Convert the Months column to datetime format

In [ ]:
data = (
    data.groupby(["Department_ID", "Months"])["Expenses"].sum().reset_index()
)  # Group by Department_ID and Months and sum the Expenses so that we get the total expenses for each department in each month

In [ ]:
pivoted_df = data.pivot(
    index="Months", columns="Department_ID", values="Expenses"
).reset_index()  # Pivot the data so that we have the months as the index and the Department_ID as the columns

In [ ]:
fig = px.line(
    pivoted_df,
    x="Months",
    y=pivoted_df.columns,
    hover_data={"Months": "|%B %d, %Y"},
    title="Expenses by Department",
)
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.show()

In [ ]:
department_id = 3100  # Department ID for which we want to forecast the expenses

data = pivoted_df[["Months", department_id]].rename(
    columns={"Months": "ds", department_id: "y"}
)  # Select the data for the department_id and rename the columns to 'ds' and 'y' as required by Prophet

In [ ]:
fig = px.line(data, x="ds", y="y")
fig.update_layout(title="Expenses of Department ID " + str(department_id))
fig.show()

In [ ]:
model = Prophet()  # Initialize Prophet model
model.fit(data)  # Fit the model
future = model.make_future_dataframe(
    periods=12, freq="M"
)  # Extend 12 months into the future
forecast = model.predict(future)  # Make predictions

In [ ]:
future["y"] = data["y"]  # Add the actual data to the future dataframe
future["yhat"] = forecast["yhat"]  # Add the predicted data to the future dataframe

In [ ]:
fig = px.line(
    future,
    x="ds",
    y="y",
    title="Expenses of Department ID " + str(department_id),
    labels={"y": "Expenses", "ds": "Months"},
)

fig.add_trace(
    go.Line(x=future["ds"], y=future["yhat"], mode="lines", name="Predicted Expenses")
)
fig.add_scatter(
    x=future["ds"],
    y=future["y"],
    mode="markers",
    marker=dict(color="blue"),
    name="Actual Expenses",
)
fig.add_scatter(
    x=future["ds"],
    y=future["yhat"],
    mode="markers",
    marker=dict(color="red"),
    name="Predicted Expenses",
)
fig.update_layout(xaxis_title="Months", yaxis_title="Expenses", hovermode="closest")

fig.show()